In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
import yaml
from models.transformer import Transformer
from dataloader import SourceTargetDataset
from train import Trainer

# Model

In [ ]:
with open("configs/config.yml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
data = SourceTargetDataset(**cfg["dataset"])

In [ ]:
model = Transformer(data.voc_src_len, data.voc_tgt_len, **cfg["model"])

In [ ]:
trainer = Trainer(**cfg["train"])
_, _ = trainer.fit(model, data)

# Inference

## Custom sentence

In [ ]:
sentence = "hey ! how are you ?"
sentence = sentence.split()
inp_pad = data.max_len - len(sentence)
sentence = sentence + ["<PAD>"] * inp_pad
inp = data._token_to_idx(sentence).reshape(1, -1)

## Sentence from training

In [ ]:
dl = iter(DataLoader(data))

In [ ]:
inp, out, inp_pad, out_pad, src, tgt = next(dl)
print(data._idx_to_token(inp, is_src=True))
print(data._idx_to_token(out))

## Inference

Initialize 'out' to the start-of-sentence (SOS)

In [ ]:
model.eval()
c_sos = data.sos.expand(1, -1)
out = torch.cat([c_sos], 1)
print(out)
print(data._idx_to_token(out))

Run inference of the sentence translated so far and append new prediction to the end of the sentence.

Select the predicted word as the one with highest probability. Change 'PICK_TOP' to select another ones.

In [ ]:
PICK_TOP = 0

R = model(inp, out, [inp_pad], [0])
last_pred = R[0,-1]
values, indices = torch.topk(last_pred, k=5)
probs = values.detach().numpy()
r = torch.tensor([indices[PICK_TOP]])
out = torch.cat([out, r.reshape(1, 1)], 1)
print("Top best values:")
print([(data.voc_tgt[x], p) for x, p in zip(indices, probs)])
print()
print("Current output:")
print(data._idx_to_token(out))